In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
  !pip install anndata==0.7.4 scanpy==1.6.0 matplotlib_scalebar==0.6.2 pandas==1.1.* tifffile==2020.* zenodo_get

In [ ]:
%%time
if IN_COLAB:
  #  Retrieve files if in google colab
  # -> This will take ca 30 minutes!
  !python -m zenodo_get -o zipdata 4288515
  !unzip -n -d data zipdata/oexp_analysis_export_v3.zip 

In [ ]:
# Python 3.8
import pathlib
import scanpy as sp # '1.6.0'
import anndata as anndata # '0.7.4'

import numpy as np # '1.19.1'
import pandas as pd # '1.1.2'
import copy

import matplotlib.pyplot as plt # '3.3.2'
from matplotlib import colors
from matplotlib_scalebar.scalebar import ScaleBar # '0.6.2'

import tifffile # '2020.9.22'

import functools

# Data export exploration

This should be a simple example how to load and visualize the exported overexpression dataset
found at: xx


This uses only 'standard' packages as well as Scanpy.
Please read the README of the export to find precise information on the data structure.


In [ ]:
class C:
    # Folder containing the unziped dataset downloaded from zenodo
    # adapt if snakemake is not used
    fol_export = pathlib.Path('data') if IN_COLAB else pathlib.Path(snakemake.input.fol_export)
    fn_cell_x = fol_export / 'cell_X.csv'
    fn_cell_obs = fol_export / 'cell_obs.csv'
    fn_cell_var = fol_export / 'cell_var.csv'
    
    fn_nuc_x = fol_export / 'nucleiexp_X.csv'
    fn_nuc_obs = fol_export / 'nucleiexp_obs.csv'
    fn_nuc_var = fol_export / 'nucleiexp_var.csv'
    
    fn_cyto_x = fol_export / 'cyto_X.csv'
    fn_cyto_obs = fol_export / 'cyto_obs.csv'
    fn_cyto_var = fol_export / 'cyto_var.csv'
    
    fn_image_meta = fol_export / 'image_meta.csv'
    fol_image = fol_export / 'images'
    fol_masks = fol_export / 'masks'
    
    fn_rel_cell_nuc = fol_export / 'relations_cell_nucleiexp.csv'
    fn_rel_cell_cyto = fol_export / 'relations_cell_cyto.csv'

Some helper scripts

-> could be also put in a library.

In [ ]:
def load_anndata(fn_x, fn_obs, fn_var):
    """
    Helper function to initialize an anndata object using csv files.
    :param fn_x: path to data csv (X)
    :param fn_obs: path to obs csv
    :param fn_var: path to var csv
    
    :return: the loaded anndata
    """
    dat_obs = pd.read_csv(fn_obs)
    dat_var = pd.read_csv(fn_var)
    dat_x = pd.read_csv(fn_x, delimiter=',', header=None)
    return anndata.AnnData(dat_x.values, obs=dat_obs, var=dat_var)
    
def transform_anndata(ad):
    # In a real analysis it may not be suitable to apply a transformation
    # to all variable, e.g. for interquartile range or other features
    # Log transform might be not optimal.
    fil = ad.var.eval("measurement_type == 'Intensity'")
    ad.X[:,fil] = np.log10(ad.X[:, fil]+0.1)
    
def map_series_on_mask(mask, label, iterable):
    """
    Maps values on a mask
    :param mask: A mask where label!=0 equals the object number
    :param iterable: an iterable
    :param label: labels of same lenght than iterables
    :return: the mapped image
    """
    # make a dict

    labeldict = np.empty(mask.max() + 1)
    labeldict[:] = np.NaN

    for lab, val in zip(label, iterable):
        labeldict[int(lab)] = val

    out_img = labeldict[mask.flatten()]
    out_img = np.reshape(out_img, mask.shape)
    out_img = np.ma.array(out_img, mask=mask == 0)
    return out_img

def plot_mask_contour(mask, ax=None, linewidths=0.5, linestyles=':', color='Gray', alpha=1):
    """
    Adds background mask contour
    """
    if ax is None:
        fig = plt.figure(figsize=(20,20))
        ax = plt.gca()
    ax.contour(mask, [0,0.5],colors=[color],linewidths=linewidths, linestyles=linestyles,alpha=alpha)
    return ax

def plot_heatmask(img, *, cmap=None, cmap_mask=None, cmap_mask_alpha=0.3, colorbar=True, ax=None,
                  bad_color='k', bad_alpha=1, crange=None, norm=None):
    """
    Plots an image with nice defaults for masked pixels.
    """
    if cmap is None:
        cmap = plt.cm.viridis
    cmap = copy.copy(cmap)
    cmap.set_bad(bad_color, bad_alpha)
    if ax is None:
        plt.close()
        fig, ax = plt.subplots(1, 1)
    else:
        fig = ax.get_figure()

    cax = ax.imshow(img, cmap=cmap, interpolation="nearest", norm=norm)
    if colorbar:
        fig.colorbar(cax)

    if crange is not None:
        cax.set_clim(crange[0], crange[1])

    if hasattr(img, "mask"):
        mask_img = np.isnan(img)
        if np.any(mask_img):
            mask_img = np.ma.array(mask_img, mask=img.mask | (mask_img == False))
            if cmap_mask is None:
                cmap_mask = "Greys"
            ax.imshow(
                mask_img == 1,
                cmap=cmap_mask,
                alpha=cmap_mask_alpha,
                interpolation="nearest",
            )
    ax.axis('off')
    return ax
    
def add_scalebar(
    ax, resolution=0.000001, location=4, color="white", pad=0.5, frameon=False, **kwargs
):
    """
    Adds a scalebar
    """
    scalebar = ScaleBar(
        resolution, location=location, color=color, pad=pad, frameon=frameon, **kwargs
    )  # 1 pixel = 0.2 meter
    ax.add_artist(scalebar)


def adapt_ax_clims(axs):
    """
    Adapts color axes limits such that they are shared by the all images
    """
    caxs = [ax.images[0] for ax in axs if len(ax.images) > 0]
    clims = [cax.get_clim() for cax in caxs]
    clims = [c for c in clims if c != (True, True)]
    clim_all = [f(c) for f, c in zip([np.min, np.max], zip(*clims))]
    for cax in caxs:
        cax.set_clim(clim_all)
        
class IoHelper:
    """
    Helper class for lazy image and mask IO
    """
    def __init__(self, dat_img, fol_images, fol_masks):
        self.dat_img = dat_img
        self.fol_masks = fol_masks
        self.fol_images = fol_images
        
    @functools.lru_cache()
    def get_mask(self, imid, object_type='cell'):
        img = self.dat_img.query(f'image_id == {imid}')[f'mask_filename_{object_type}'].iloc[0]
        return tifffile.imread(self.fol_masks / img)


    @functools.lru_cache()
    def get_image(self, imid, stack_name='FullStackComp'):
        img = self.dat_img.query(f'image_id == {imid}')[f'image_stack_filename_{stack_name}'].iloc[0]
        return tifffile.imread(self.fol_images / img, out='memmap')


    def get_image_channel(self, imid, plane_number, stack_name='FullStackComp'):
        img = self.get_image(imid, stack_name)
        if (img.shape[2] == self.dat_img.query(f'image_id == {imid}')['image_shape_w'].iloc[0]):
            return img[plane_number-1, :, :].squeeze()
        else:
            return img[:, :, plane_number-1].squeeze()
    
    def plot_imag_ad(self, ad, channel_name,
                     measurement_name='MeanIntensityComp',
                     stack_name='FullStackFiltered',
                     figsize=None, add_colorbar=True):
        """
        Plots all objects contained in the anndata, plotting the individual images as columns.
        
        :param ad: an anndata with obs variables: 'image_id', 'object_type', 'object_number'
        :param chan: The channel (metal) to plot
        :param figsize: the output image size
        
        :returns: the figure object
        """
        imgids = ad.obs['image_id'].unique()
        fig, axs = plt.subplots(ncols=len(imgids), figsize=figsize)
        try:
            len(axs)
        except:
            axs = [axs]
        
        fil = ad.var.eval(f'channel_name == "{channel_name}" and measurement_name == "{measurement_name}" and '
                      f'stack_name == "{stack_name}"')
        
        assert sum(fil) == 1
        for ax, ((obj_type, imid), dat) in zip(axs, ad.obs.groupby(['object_type','image_id'])):
            mask = self.get_mask(imid, object_type=obj_type)
            values = ad[dat.index, :][:, fil].X.squeeze()
            labels = dat['object_number']
            img = map_series_on_mask(mask, labels, values)
            
            colorbar = (ax == axs[-1]) and add_colorbar
            plot_heatmask(img, ax=ax, colorbar=colorbar)
            ax.axis('off')
        adapt_ax_clims(axs)
        return fig
        
    


Read the image metadta and initialize the IO handler

-> This allows to conveniently retrieve images and maks.

In [ ]:
dat_image = pd.read_csv(C.fn_image_meta)

In [ ]:
dat_image.iloc[0,:]

In [ ]:
io = IoHelper(dat_image, fol_images=C.fol_image, fol_masks=C.fol_masks)

Load the cell anndata

-> All analyses in the paper were only done based on cellular readouts.

This also transforms the data using: log10(x+0.1)

In [ ]:
ad_cell = load_anndata(C.fn_cell_x, C.fn_cell_obs, C.fn_cell_var)
transform_anndata(ad_cell)

In [ ]:
ad_cell.obs.head()

In [ ]:
ad_cell.obs.describe()

In [ ]:
ad_cell.var

In [ ]:
ad_cell.var.measurement_name.unique()

Add a 'plot name' that combines the metal with a human readable name

In [ ]:
ad_cell.var['plot_name'] = (ad_cell.var
                            .apply(lambda row: f'{row["goodname"]} - {row["channel_name"]}', axis=1)
                            .astype(pd.CategoricalDtype())
                           )

In [ ]:
ad_cell.var['plot_name'] 

Show how this could be visualized using eg a PCA

In [ ]:
ad_cell.var_names = ad_cell.var.measurement_id.astype(str)

In [ ]:
ad_cell.obs.iloc[0,:]

In [ ]:
dat_image.iloc[0,:]

In [ ]:
ad_cell.obs = ad_cell.obs.reset_index().merge(dat_image[['image_id', 'condition_name']], how='left',
                           ).set_index('index')

In [ ]:
fil_main = ad_cell.var.eval('working == 1 and measurement_name == "MeanIntensityComp"'
                            'and stack_name == "FullStackFiltered"')
tmp_ad = ad_cell[:, fil_main]
sp.preprocessing.pca(tmp_ad)
ad_cell.obsm = tmp_ad.obsm

In [ ]:
sp.pl.pca(ad_cell, color='modelfitcond_v1')

In [ ]:
fil = ad_cell.var.eval(f'measurement_name == "MeanIntensityComp" and channel_name in ["Er167", "Tm169"]')
axs = sp.pl.pca(ad_cell, color=ad_cell.var.index[fil], show=False)
for ax, tit in zip(axs, ad_cell.var.plot_name[fil]):
    ax.set_title(tit)
plt.show()

In [ ]:
len(ad_cell.var.plot_name.cat.categories)

Visualize the data as a heatmap

In [ ]:
fil = ad_cell.var.eval('working == 1 and measurement_name == "MeanIntensityComp"'
                            'and stack_name == "FullStackFiltered"')
axs = sp.pl.heatmap(ad_cell, var_names=ad_cell.var.index[fil], log=False, groupby='modelfitcond_v1',
             show_gene_labels=True, standard_scale='var', show=False
               )

axs['heatmap_ax'].set_xticklabels(ad_cell.var.plot_name[fil])

Show how the data could be visualized using a heatplot, mapping the average intensities on a mask


-> Cells which were filtered out during QC are displayed as grey

In [ ]:
fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id.isin(ad_cell.obs.image_id.unique()[:10]) , :],
                      'Ir193', figsize=(15,20))

Show how raw images could be visualized

In [ ]:
imid = 123

In [ ]:
fil = ad_cell.var.eval(f'channel_name == "Tm169" and stack_name == "FullStackFiltered"')
img = io.get_image_channel(imid, ad_cell.var.ref_plane_number[fil][0])
ax= plot_heatmask(img,norm=colors.PowerNorm(gamma=1./2.)) # This plots the image with sqareroot transformed values
add_scalebar(ax)
plot_mask_contour(io.get_mask(imid), ax=ax)

In [ ]:
def highlight_classes(imid, ad, ax):
  """
  Helper function:
  Highlight overexpressing cells in green, doubt in red (not clear if overexpressing or
  neighours of overexpressing cells) and neighbours in blue (clearly not overexpressing but
  neighbour of overexpressing)

  """
  tad = ad[ad.obs.image_id == imid, :]
  for col, cls in (['green', 'oexp'], ['blue', 'oexp-NB'], ['red', 'doubt']):
    fil = tad.obs.eval(f"modelfitcond_v1 == '{cls}'")
    plot_mask_contour(np.isin(io.get_mask(imid), tad.obs.object_number.loc[fil]),
                      ax=ax, color=col, alpha=0.8)
  return ax


In [ ]:
ax = highlight_classes(imid, ad_cell, ax)
ax.get_figure()

In [ ]:
fil = ad_cell.var.eval(f'channel_name == "GFP" and stack_name == "IfStack"')
img = io.get_image_channel(imid, ad_cell.var.ref_plane_number[fil][0], stack_name="IfStack")
ax= plot_heatmask(img, norm=colors.PowerNorm(gamma=1./2.))
add_scalebar(ax)
plot_mask_contour(io.get_mask(imid), ax=ax)

In [ ]:
ax = highlight_classes(imid, ad_cell, ax)
ax.get_figure()

In [ ]:
fil = ad_cell.var.eval(f'channel_name == "DAPI" and stack_name == "IfStack"')
img = io.get_image_channel(imid, ad_cell.var.ref_plane_number[fil][0], stack_name="IfStack")
ax= plot_heatmask(img, norm=colors.PowerNorm(gamma=1./2.))
add_scalebar(ax)
plot_mask_contour(io.get_mask(imid), ax=ax, color='red')

The same image as heatmask

In [ ]:
fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id == imid, :], 'Tm169')
add_scalebar(fig.axes[0])

In [ ]:
fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id == imid, :], 'GFP',
                     measurement_name='MeanIntensity',
                     stack_name='IfStack')
add_scalebar(fig.axes[0])

fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id == imid, :],
                      'Tm169',
                     measurement_name='NbMeanMeanIntensityComp',
                     stack_name='FullStackFiltered')
add_scalebar(fig.axes[0])

In [ ]:
fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id == imid, :],
                     channel_name='object', measurement_name='dist-rim',
                     stack_name='ObjectStack')
add_scalebar(fig.axes[0])

In [ ]:
fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id == imid, :],
                     channel_name='object', measurement_name='Center_X',
                     stack_name='ObjectStack')
add_scalebar(fig.axes[0])

In [ ]:
fig = io.plot_imag_ad(ad_cell[ad_cell.obs.image_id == imid, :],
                     channel_name='object', measurement_name='Area',
                     stack_name='ObjectStack')
add_scalebar(fig.axes[0])

## Also load nuclear and cytoplasmic data

(This was not used in the paper)

### Load and plot nuclear data

In [ ]:
ad_nuc = load_anndata(C.fn_nuc_x, C.fn_nuc_obs, C.fn_nuc_var)
transform_anndata(ad_nuc)
ad_nuc.var_names = ad_nuc.var.measurement_id.astype(str)

Get relation with cell ids

In [ ]:
rel_cell_nuc = pd.read_csv(C.fn_rel_cell_nuc)
rel_cell_nuc.head(1)

Identify nuclei of overexpressing cells via relations

In [ ]:
ad_nuc.obs = (
    rel_cell_nuc
    # sometimes a nuclei can belong to two cells...
    .drop_duplicates('object_id_nucleiexp')
     .merge(ad_cell.obs[['object_id', 'modelfitcond_v1']], left_on='object_id_cell', right_on='object_id')
     .drop(columns=['object_id'])
     .merge(ad_nuc.obs.reset_index(), left_on='object_id_nucleiexp', right_on='object_id', how='right')
     .set_index('index')
)

Show example image nuclei

In [ ]:
fig = io.plot_imag_ad(ad_nuc[ad_nuc.obs.image_id == imid, :], 'Tm169')
add_scalebar(fig.axes[0])

Also do pca and heamap for nuclei

In [ ]:
fil_main = ad_nuc.var.eval('working == 1 and measurement_name == "MeanIntensityComp"'
                            'and stack_name == "FullStackFiltered"')
tmp_ad = ad_nuc[:, fil_main]
sp.preprocessing.pca(tmp_ad)
ad_nuc.obsm = tmp_ad.obsm

In [ ]:
ad_nuc.obs = ad_nuc.obs.reset_index().merge(dat_image[['image_id', 'condition_name']], how='left',
                           ).set_index('index')

In [ ]:
sp.pl.pca(ad_nuc, color='modelfitcond_v1')

In [ ]:
fil = ad_nuc.var.eval(f'measurement_name == "MeanIntensityComp" and channel_name in ["Er167", "Tm169"]')
axs = sp.pl.pca(ad_nuc, color=ad_nuc.var.index[fil], show=False)
for ax, tit in zip(axs, ad_nuc.var.goodname[fil]):
    ax.set_title(tit)
plt.show()

In [ ]:
fil = fil_main | (ad_nuc.var.measurement_name == 'Area')
axs = sp.pl.heatmap(ad_nuc, var_names=ad_nuc.var.index[fil], log=False, groupby='modelfitcond_v1',
             show_gene_labels=True, standard_scale='var', show=False
               )

axs['heatmap_ax'].set_xticklabels(ad_nuc.var.goodname[fil])

### Load cytoplasm data

In [ ]:
ad_cyto = load_anndata(C.fn_cyto_x, C.fn_cyto_obs, C.fn_cyto_var)
transform_anndata(ad_cyto)
ad_cyto.var_names = ad_cyto.var.measurement_id.astype(str)

Match cytopasm 

Get relation with cell ids

In [ ]:
rel_cell_cyto = pd.read_csv(C.fn_rel_cell_cyto)
rel_cell_cyto.head(1)

Identify cytoplasm of overexpressing cells via relations

In [ ]:
ad_cyto.obs = (
    rel_cell_cyto
    # sometimes a cytoplasm can belong to two cells...
    .drop_duplicates('object_id_cyto')
     .merge(ad_cell.obs[['object_id', 'modelfitcond_v1']], left_on='object_id_cell', right_on='object_id')
     .drop(columns=['object_id'])
     .merge(ad_cyto.obs.reset_index(), left_on='object_id_cyto', right_on='object_id', how='right')
     .set_index('index')
)

Show example image cytoplasm

In [ ]:
fig = io.plot_imag_ad(ad_cyto[ad_cyto.obs.image_id == imid, :], 'Tm169')
add_scalebar(fig.axes[0])

Plot heatmap and PCA

In [ ]:
ad_cyto.obs = ad_cyto.obs.reset_index().merge(dat_image[['image_id', 'condition_name']], how='left',
                           ).set_index('index')

In [ ]:
fil_main = ad_cyto.var.eval('working == 1 and measurement_name == "MeanIntensityComp"'
                            'and stack_name == "FullStackFiltered"')
tmp_ad = ad_cyto[:, fil_main]
sp.preprocessing.pca(tmp_ad)
ad_cyto.obsm = tmp_ad.obsm


In [ ]:
sp.pl.pca(ad_cyto, color='modelfitcond_v1')

In [ ]:
fil = ad_cyto.var.eval(f'measurement_name == "MeanIntensityComp" and channel_name in ["Er167", "Tm169"]')
axs = sp.pl.pca(ad_cyto, color=ad_cyto.var.index[fil], show=False)
for ax, tit in zip(axs, ad_cyto.var.goodname[fil]):
    ax.set_title(tit)
plt.show()

In [ ]:
fil = fil_main | (ad_cyto.var.measurement_name == 'Area')
axs = sp.pl.heatmap(ad_nuc, var_names=ad_nuc.var.channel_name[fil], log=False, groupby='modelfitcond_v1',
             gene_symbols='channel_name',show_gene_labels=True, standard_scale='var', show=False
               )

axs['heatmap_ax'].set_xticklabels(ad_nuc.var.goodname[fil])

Scatterplot example

This is a very simple example how a scatterplot of markers could be done

In [ ]:
x = ad_cell.var.query(f'measurement_name == "MeanIntensityComp" and channel_name == "Tm169"').index[0]
y = ad_cell.var.query(f'measurement_name == "MeanIntensityComp" and channel_name == "Er167"').index[0]
obs_col = 'modelfitcond_v1'
xlab, ylab = (ad_cell.var.goodname[m] for m in (x,y))

ncat = len(ad_cell.obs[obs_col].cat.categories)
fig, axs = plt.subplots(ncols=ncat,sharex=True, sharey=True, figsize=(20,5))

for ax, (group, dat) in zip(axs, ad_cell.obs.groupby(obs_col)):
  ax.hexbin(ad_cell[dat.index, :].obs_vector(x), 
            ad_cell[dat.index, :].obs_vector(y),)
  ax.set_title(group)
  ax.set_xlabel(xlab)
  ax.set_ylabel(ylab)
  ax.set_facecolor(plt.cm.viridis.colors[0])
